# What's in this exercise?

1) Read raw data, augment with derived attributes, augment with reference data & persist<BR> 
2) Create external unmanaged Hive tables<BR>
3) Create statistics for tables

In [267]:
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType, IntegerType,LongType,FloatType,DoubleType, TimestampType

### 1.  Execute notebook with common/reusable functions

In [268]:
%run "../../../../01-General/2-CommonFunctions.ipynb"

Code from file 'file:///home/dinhnn/git/personal/Databricks-NYC-Taxi/Workspace/01-General/2-CommonFunctions.ipynb':
 import os
import math
import glob
import re
prqShrinkageFactor = 0.19 #We found a saving in space of 81% with Parquet
def analyzeTables(databaseAndTable):
  try:
    print("Table: " + databaseAndTable)
    print("....refresh table")
    sql("REFRESH TABLE " + databaseAndTable)
    print("....analyze table")
    sql("ANALYZE TABLE " + databaseAndTable + " COMPUTE STATISTICS")
    print("....done")
  except Exception as e:
    return e
def calcOutputFileCountTxtToPrq(srcDataFile, targetedFileSizeMB):
  try:
    estFileCount = int(math.floor((os.path.getsize(srcDataFile) * prqShrinkageFactor) / (targetedFileSizeMB * 1024 * 1024)))
    if(estFileCount == 0):
      return 1 
    else:
      return estFileCount
  except Exception as e:
    return e
#Delete residual files from job operation (_SUCCESS, _start*, _committed*)
#Should be called with '/dbfs/mnt/...'
def recursivelyD

### 2.  Read raw, augment, persist as parquet

In [ ]:
# Non optimized

# curatedDF = sql("""
#   select distinct t.taxi_type,
#       t.vendor_id as vendor_id,
#       t.pickup_datetime,
#       t.dropoff_datetime,
#       t.store_and_fwd_flag,
#       t.rate_code_id,
#       t.pickup_location_id,
#       t.dropoff_location_id,
#       t.pickup_longitude,
#       t.pickup_latitude,
#       t.dropoff_longitude,
#       t.dropoff_latitude,
#       t.passenger_count,
#       t.trip_distance,
#       t.fare_amount,
#       t.extra,
#       t.mta_tax,
#       t.tip_amount,
#       t.tolls_amount,
#       t.improvement_surcharge,
#       t.total_amount,
#       t.payment_type,
#       t.trip_year,
#       t.trip_month,
#       v.abbreviation as vendor_abbreviation,
#       v.description as vendor_description,
#       tm.month_name_short,
#       tm.month_name_full,
#       pt.description as payment_type_description,
#       rc.description as rate_code_description,
#       tzpu.borough as pickup_borough,
#       tzpu.zone as pickup_zone,
#       tzpu.service_zone as pickup_service_zone,
#       tzdo.borough as dropoff_borough,
#       tzdo.zone as dropoff_zone,
#       tzdo.service_zone as dropoff_service_zone,
#       year(t.pickup_datetime) as pickup_year,
#       month(t.pickup_datetime) as pickup_month,
#       day(t.pickup_datetime) as pickup_day,
#       hour(t.pickup_datetime) as pickup_hour,
#       minute(t.pickup_datetime) as pickup_minute,
#       second(t.pickup_datetime) as pickup_second,
#       date(t.pickup_datetime) as pickup_date,
#       year(t.dropoff_datetime) as dropoff_year,
#       month(t.dropoff_datetime) as dropoff_month,
#       day(t.dropoff_datetime) as dropoff_day,
#       hour(t.dropoff_datetime) as dropoff_hour,
#       minute(t.dropoff_datetime) as dropoff_minute,
#       second(t.dropoff_datetime) as dropoff_second,
#       date(t.dropoff_datetime) as dropoff_date
#   from 
#     `synapse_nyc_reference`.`nyctaxi`.`yellow_taxi_trips_raw` t
#     left join `synapse_nyc_reference`.`nyctaxi`.vendor_lookup v 
#       on (t.vendor_id = v.abbreviation)
#     left join  `synapse_nyc_reference`.`nyctaxi`.trip_month_lookup tm 
#       on (t.trip_month = tm.trip_month)
#     left join  `synapse_nyc_reference`.`nyctaxi`.payment_type_lookup pt 
#       on (t.payment_type = pt.abbreviation)
#     left join  `synapse_nyc_reference`.`nyctaxi`.rate_code_lookup rc 
#       on (t.rate_code_id = rc.rate_code_id)
#     left join  `synapse_nyc_reference`.`nyctaxi`.taxi_zone_lookup tzpu 
#       on (t.pickup_location_id = tzpu.location_id)
#     left join  `synapse_nyc_reference`.`nyctaxi`.taxi_zone_lookup tzdo 
#       on (t.dropoff_location_id = tzdo.location_id)
# WHERE t.trip_year < 2015
# UNION ALL
#   select distinct t.taxi_type,
#       t.vendor_id as vendor_id,
#       t.pickup_datetime,
#       t.dropoff_datetime,
#       t.store_and_fwd_flag,
#       t.rate_code_id,
#       t.pickup_location_id,
#       t.dropoff_location_id,
#       t.pickup_longitude,
#       t.pickup_latitude,
#       t.dropoff_longitude,
#       t.dropoff_latitude,
#       t.passenger_count,
#       t.trip_distance,
#       t.fare_amount,
#       t.extra,
#       t.mta_tax,
#       t.tip_amount,
#       t.tolls_amount,
#       t.improvement_surcharge,
#       t.total_amount,
#       t.payment_type,
#       t.trip_year,
#       t.trip_month,
#       v.abbreviation as vendor_abbreviation,
#       v.description as vendor_description,
#       tm.month_name_short,
#       tm.month_name_full,
#       pt.description as payment_type_description,
#       rc.description as rate_code_description,
#       tzpu.borough as pickup_borough,
#       tzpu.zone as pickup_zone,
#       tzpu.service_zone as pickup_service_zone,
#       tzdo.borough as dropoff_borough,
#       tzdo.zone as dropoff_zone,
#       tzdo.service_zone as dropoff_service_zone,
#       year(t.pickup_datetime) as pickup_year,
#       month(t.pickup_datetime) as pickup_month,
#       day(t.pickup_datetime) as pickup_day,
#       hour(t.pickup_datetime) as pickup_hour,
#       minute(t.pickup_datetime) as pickup_minute,
#       second(t.pickup_datetime) as pickup_second,
#       date(t.pickup_datetime) as pickup_date,
#       year(t.dropoff_datetime) as dropoff_year,
#       month(t.dropoff_datetime) as dropoff_month,
#       day(t.dropoff_datetime) as dropoff_day,
#       hour(t.dropoff_datetime) as dropoff_hour,
#       minute(t.dropoff_datetime) as dropoff_minute,
#       second(t.dropoff_datetime) as dropoff_second,
#       date(t.dropoff_datetime) as dropoff_date
#   from 
#     `synapse_nyc_reference`.`nyctaxi`.`yellow_taxi_trips_raw` t
#     left join `synapse_nyc_reference`.`nyctaxi`.vendor_lookup v 
#       on (t.vendor_id = v.vendor_id)
#     left join  `synapse_nyc_reference`.`nyctaxi`.trip_month_lookup tm 
#       on (t.trip_month = tm.trip_month)
#     left join  `synapse_nyc_reference`.`nyctaxi`.payment_type_lookup pt 
#       on (t.payment_type = pt.payment_type)
#     left join  `synapse_nyc_reference`.`nyctaxi`.rate_code_lookup rc 
#       on (t.rate_code_id = rc.rate_code_id)
#     left join  `synapse_nyc_reference`.`nyctaxi`.taxi_zone_lookup tzpu 
#       on (t.pickup_location_id = tzpu.location_id)
#     left join  `synapse_nyc_reference`.`nyctaxi`.taxi_zone_lookup tzdo 
#       on (t.dropoff_location_id = tzdo.location_id)
# WHERE t.trip_year >= 2015
#   """)

# curatedDFConformed = (curatedDF.withColumn("temp_vendor_id", col("vendor_id").cast("integer")).drop("vendor_id").withColumnRenamed("temp_vendor_id", "vendor_id").withColumn("temp_payment_type", col("payment_type").cast("integer")).drop("payment_type").withColumnRenamed("temp_payment_type", "payment_type"))

#Save as parquet, partition by year and month
#curatedDFConformed.coalesce(15).write.partitionBy("trip_year", "trip_month").parquet(destDataDirRoot)

In [269]:
DATABRICKS_HOST = dbutils.secrets.get(scope="databricks-warehouse", key="sql-host").replace("https://", "")
DATABRICKS_HTTP_PATH = f"/sql/1.0/warehouses/{dbutils.secrets.get(scope="databricks-warehouse", key="warehouseid")}"
DATABRICKS_TOKEN = dbutils.secrets.get(scope="databricks-warehouse", key="sql-token")

# JDBC connection URL for Databricks
databricks_jdbc_url = f"jdbc:databricks://{DATABRICKS_HOST}:443;transportMode=http;ssl=1;httpPath={DATABRICKS_HTTP_PATH};AuthMech=3;UID=token;PWD={DATABRICKS_TOKEN}"

In [270]:
destDataDirRoot = f"gs://nyctaxi-silver/nyctaxi/transform/yellow-taxi/" #Root dir for consumable data

In [271]:
# Get the main fact table with optimized reading strategy using Databricks JDBC with a single partition key
def optimized_jdbc_read(sql: str = "", table: str = "", partition_filter: str = "", partition_column: str = "", num_partitions=200, jdbc_url=databricks_jdbc_url):
    """
    Read a table from Databricks using JDBC with optimized settings and a single partition key
    
    Args:
        table: Full table name including catalog and schema (e.g., "catalog.schema.table") or temp table sql (for generate partition id column)
        needed_columns: List of columns to select (None for all)
        partition_filter: SQL filter expression for partition pruning
        partition_keys: List of column names to use for partitioning (only the first one will be used)
        num_partitions: Number of partitions to use (default: 200)
        jdbc_url: JDBC connection URL for Databricks
    
    Returns:
        DataFrame with optimized reading settings
        
    Raises:
        ValueError: If multiple partition keys are provided
    """
    # Define JDBC connection properties
    jdbc_properties = {
        "driver": "com.databricks.client.jdbc.Driver",
        "fetchsize": "1000000"  # Fetch size for better performance
    }
    
    # com.databricks.client.jdbc.Driver
    
    # If no partition keys provided, use default approach
    if not partition_column:
      return spark.read.format("jdbc") \
                          .option("driver", "com.databricks.client.jdbc.Driver") \
                          .option("url", jdbc_url) \
                          .option("dbtable", f"({sql}) as trip_data") \
                          .option("fetchsize", "1000000") \
                          .load()
    
    # Get min and max values for the partition column
    min_max_query = f"SELECT MIN({partition_column}) as min_val, MAX({partition_column}) as max_val FROM {f"({table})" if "select" in table.lower() else table}"
    if partition_filter:
        min_max_query += f" WHERE {partition_filter}"
        
    # Execute query to get bounds
    bounds_df = spark.read.format("jdbc") \
                          .option("driver", "com.databricks.client.jdbc.Driver") \
                          .option("url", jdbc_url) \
                          .option("dbtable", f"({min_max_query}) as min_max_bounds") \
                          .option("fetchsize", "1000000") \
                          .load()
    
    # Extract bounds
    bounds_row = bounds_df.first()
    lower_bound = bounds_row["min_val"]
    upper_bound = bounds_row["max_val"]
    
    # Ensure valid bounds (avoid division by zero)
    if lower_bound == upper_bound:
        upper_bound = lower_bound + 1
    
    # Read with numeric partitioning
    return spark.read.format("jdbc") \
                     .option("driver", "com.databricks.client.jdbc.Driver") \
                     .option("url", jdbc_url) \
                     .option("dbtable", f"({sql}) as tmp") \
                     .option("numPartitions", num_partitions) \
                     .option("partitionColumn", partition_column) \
                     .option("lowerBound", lower_bound) \
                     .option("upperBound", upper_bound) \
                     .option("fetchsize", "100000") \
                     .load()

In [272]:
yellow_taxi_transform_sql = """

SELECT /*+ BROADCAST(v), BROADCAST(tm), BROADCAST(pt), BROADCAST(rc), BROADCAST(tzpu), BROADCAST(tzdo) */ DISTINCT
  t.taxi_type,
  t.vendor_id,
  t.pickup_datetime,
  t.dropoff_datetime,
  t.store_and_fwd_flag,
  t.rate_code_id,
  t.pickup_location_id,
  t.dropoff_location_id,
  t.pickup_longitude,
  t.pickup_latitude,
  t.dropoff_longitude,
  t.dropoff_latitude,
  t.passenger_count,
  t.trip_distance,
  t.fare_amount,
  t.extra,
  t.mta_tax,
  t.tip_amount,
  t.tolls_amount,
  t.improvement_surcharge,
  t.total_amount,
  t.payment_type,
  t.trip_year,
  t.trip_month,
  v.abbreviation AS vendor_abbreviation,
  v.description AS vendor_description,
  tm.month_name_short,
  tm.month_name_full,
  pt.description AS payment_type_description,
  rc.description AS rate_code_description,
  tzpu.borough AS pickup_borough,
  tzpu.zone AS pickup_zone,
  tzpu.service_zone AS pickup_service_zone,
  tzdo.borough AS dropoff_borough,
  tzdo.zone AS dropoff_zone,
  tzdo.service_zone AS dropoff_service_zone,
  year(t.pickup_datetime) AS pickup_year,
  month(t.pickup_datetime) AS pickup_month,
  day(t.pickup_datetime) AS pickup_day,
  hour(t.pickup_datetime) AS pickup_hour,
  minute(t.pickup_datetime) AS pickup_minute,
  second(t.pickup_datetime) AS pickup_second,
  date(t.pickup_datetime) AS pickup_date,
  year(t.dropoff_datetime) AS dropoff_year,
  month(t.dropoff_datetime) AS dropoff_month,
  day(t.dropoff_datetime) AS dropoff_day,
  hour(t.dropoff_datetime) AS dropoff_hour,
  minute(t.dropoff_datetime) AS dropoff_minute,
  second(t.dropoff_datetime) AS dropoff_second,
  date(t.dropoff_datetime) AS dropoff_date,
  unix_millis(t.pickup_datetime) as partition_id
FROM 
  `synapse_nyc_reference`.`nyctaxi`.yellow_taxi_trips_raw t
  LEFT OUTER JOIN `synapse_nyc_reference`.`nyctaxi`.vendor_lookup v
    ON (t.vendor_id = v.abbreviation)
  LEFT OUTER JOIN `synapse_nyc_reference`.`nyctaxi`.trip_month_lookup tm
    ON (t.trip_month = tm.trip_month)
  LEFT OUTER JOIN `synapse_nyc_reference`.`nyctaxi`.payment_type_lookup pt
    ON (t.payment_type = pt.abbreviation)
  LEFT OUTER JOIN `synapse_nyc_reference`.`nyctaxi`.rate_code_lookup rc
    ON (t.rate_code_id = rc.rate_code_id)
  LEFT OUTER JOIN `synapse_nyc_reference`.`nyctaxi`.taxi_zone_lookup tzpu
    ON (t.pickup_location_id = tzpu.location_id)
  LEFT OUTER JOIN `synapse_nyc_reference`.`nyctaxi`.taxi_zone_lookup tzdo
    ON (t.dropoff_location_id = tzdo.location_id)
WHERE
  t.trip_year < 2015

UNION ALL

SELECT /*+ BROADCAST(v), BROADCAST(tm), BROADCAST(pt), BROADCAST(rc), BROADCAST(tzpu), BROADCAST(tzdo) */ DISTINCT
  t.taxi_type,
  t.vendor_id,
  t.pickup_datetime,
  t.dropoff_datetime,
  t.store_and_fwd_flag,
  t.rate_code_id,
  t.pickup_location_id,
  t.dropoff_location_id,
  t.pickup_longitude,
  t.pickup_latitude,
  t.dropoff_longitude,
  t.dropoff_latitude,
  t.passenger_count,
  t.trip_distance,
  t.fare_amount,
  t.extra,
  t.mta_tax,
  t.tip_amount,
  t.tolls_amount,
  t.improvement_surcharge,
  t.total_amount,
  t.payment_type,
  t.trip_year,
  t.trip_month,
  v.abbreviation AS vendor_abbreviation,
  v.description AS vendor_description,
  tm.month_name_short,
  tm.month_name_full,
  pt.description AS payment_type_description,
  rc.description AS rate_code_description,
  tzpu.borough AS pickup_borough,
  tzpu.zone AS pickup_zone,
  tzpu.service_zone AS pickup_service_zone,
  tzdo.borough AS dropoff_borough,
  tzdo.zone AS dropoff_zone,
  tzdo.service_zone AS dropoff_service_zone,
  year(t.pickup_datetime) AS pickup_year,
  month(t.pickup_datetime) AS pickup_month,
  day(t.pickup_datetime) AS pickup_day,
  hour(t.pickup_datetime) AS pickup_hour,
  minute(t.pickup_datetime) AS pickup_minute,
  second(t.pickup_datetime) AS pickup_second,
  date(t.pickup_datetime) AS pickup_date,
  year(t.dropoff_datetime) AS dropoff_year,
  month(t.dropoff_datetime) AS dropoff_month,
  day(t.dropoff_datetime) AS dropoff_day,
  hour(t.dropoff_datetime) AS dropoff_hour,
  minute(t.dropoff_datetime) AS dropoff_minute,
  second(t.dropoff_datetime) AS dropoff_second,
  date(t.dropoff_datetime) AS dropoff_date,
  unix_millis(t.pickup_datetime) as partition_id
FROM 
  `synapse_nyc_reference`.`nyctaxi`.yellow_taxi_trips_raw t
  LEFT OUTER JOIN `synapse_nyc_reference`.`nyctaxi`.vendor_lookup v
    ON (t.vendor_id = v.vendor_id)
  LEFT OUTER JOIN `synapse_nyc_reference`.`nyctaxi`.trip_month_lookup tm
    ON (t.trip_month = tm.trip_month)
  LEFT OUTER JOIN `synapse_nyc_reference`.`nyctaxi`.payment_type_lookup pt
    ON (t.payment_type = pt.payment_type)
  LEFT OUTER JOIN `synapse_nyc_reference`.`nyctaxi`.rate_code_lookup rc
    ON (t.rate_code_id = rc.rate_code_id)
  LEFT OUTER JOIN `synapse_nyc_reference`.`nyctaxi`.taxi_zone_lookup tzpu
    ON (t.pickup_location_id = tzpu.location_id)
  LEFT OUTER JOIN `synapse_nyc_reference`.`nyctaxi`.taxi_zone_lookup tzdo
    ON (t.dropoff_location_id = tzdo.location_id)
WHERE
  t.trip_year >= 2015
"""

partition_sql_table = """
SELECT unix_millis(t.pickup_datetime) as partition_id FROM `synapse_nyc_reference`.`nyctaxi`.`yellow_taxi_trips_raw` t
"""


curatedDF = optimized_jdbc_read(
    sql=yellow_taxi_transform_sql,
    table=partition_sql_table,
    partition_filter="",
    partition_column="partition_id",
    num_partitions=4000
).cache()

curatedDF.count()

# # Persist the final DataFrame with an optimized storage level
# # This helps avoid recomputation and reduces shuffling in subsequent operations
# from pyspark.storagelevel import StorageLevel
# curatedDF = curatedDF.persist(StorageLevel.MEMORY_ONLY)

curatedDFConformed = (curatedDF.withColumn("temp_vendor_id", col("vendor_id").cast("integer")).drop("vendor_id").withColumnRenamed("temp_vendor_id", "vendor_id").withColumn("temp_payment_type", col("payment_type").cast("integer")).drop("payment_type").withColumnRenamed("temp_payment_type", "payment_type"))
dbutils.fs.rm(destDataDirRoot,recurse=True)
curatedDFConformed.write.format("delta").mode("overwrite").partitionBy("trip_year","trip_month").save(destDataDirRoot)

In [ ]:
curatedDF.explain(True)  # Shows the execution plan

### 3.  Define external table

In [ ]:
def create_table(schema: str, table_name: str, parquet_dir: str, location: str):
    spark.sql(f"use {schema};")
    spark.sql(f"DROP TABLE IF EXISTS {table_name};")
    spark.sql(f"CREATE TABLE IF NOT EXISTS {table_name} USING delta LOCATION '{location}/{parquet_dir}';")

In [ ]:
create_table(schema="synapse_nyc_reference.nyctaxi", table_name="yellow_taxi_trips_transform", parquet_dir="", location=destDataDirRoot)

### 4.  Explore

In [ ]:
%sql
select count(*) as trip_count from taxi_db.yellow_taxi_trips_curated

In [ ]:
%sql
select trip_year,trip_month, count(*) as trip_count from taxi_db.yellow_taxi_trips_curated group by trip_year,trip_month